In [1]:
import numpy as np
import h5py

datafile = h5py.File('SCNeuronModelCompetition.mat')
movie = datafile.get('trainingmovie_norm') # normalized movie
frhist = datafile.get('FRhist_tr') # firing rate histograms
# movie = np.array(movie).reshape(288, 150, 128, 96)

In [4]:
import keras
from keras.layers import CuDNNLSTM, Activation, Dense, BatchNormalization, Convolution2D

seq_len = movie.shape[1]
input_size = movie.shape[2]
num_layer = 3
num_hidden = 100
output_size = frhist.shape[2]

model = keras.models.Sequential()
# model.add(Convolution2D(32, 4, 4, border_mode="valid", input_shape=[1, 56,14]))
model.add(CuDNNLSTM(num_hidden, input_shape=(seq_len, input_size), return_sequences=True))

for _ in range(1, num_layer):
    model.add(BatchNormalization(momentum=0))
    model.add(Activation('relu'))
    model.add(CuDNNLSTM(num_hidden, return_sequences=True))

model.add(BatchNormalization(momentum=0))
model.add(Activation('linear'))
model.add(Dense(output_size))
model.add(Activation('softplus'))
adamopt = keras.optimizers.Adam(lr=0.001, decay=1e-7)

model.compile(optimizer=adamopt, loss='poisson')
model.summary()

early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
history = model.fit(movie, frhist, epochs=200, batch_size=32, validation_split=0.2, shuffle=True, callbacks=[early_stopping])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cu_dnnlstm_1 (CuDNNLSTM)     (None, 150, 100)          4956000   
_________________________________________________________________
batch_normalization_1 (Batch (None, 150, 100)          400       
_________________________________________________________________
activation_1 (Activation)    (None, 150, 100)          0         
_________________________________________________________________
cu_dnnlstm_2 (CuDNNLSTM)     (None, 150, 100)          80800     
_________________________________________________________________
batch_normalization_2 (Batch (None, 150, 100)          400       
_________________________________________________________________
activation_2 (Activation)    (None, 150, 100)          0         
_________________________________________________________________
cu_dnnlstm_3 (CuDNNLSTM)     (None, 150, 100)          80800     
__________

InvalidArgumentError: No OpKernel was registered to support Op 'CudnnRNN' with these attrs.  Registered devices: [CPU], Registered kernels:
  device='GPU'; T in [DT_FLOAT]

	 [[Node: cu_dnnlstm_1/CudnnRNN = CudnnRNN[T=DT_FLOAT, direction="unidirectional", dropout=0, input_mode="linear_input", is_training=true, rnn_mode="lstm", seed=87654321, seed2=0](cu_dnnlstm_1/transpose, cu_dnnlstm_1/ExpandDims_1, cu_dnnlstm_1/ExpandDims_2, cu_dnnlstm_1/concat_1)]]

Caused by op 'cu_dnnlstm_1/CudnnRNN', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/cs231n/myVE35/lib/python3.5/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/cs231n/myVE35/lib/python3.5/site-packages/traitlets/config/application.py", line 592, in launch_instance
    app.start()
  File "/home/cs231n/myVE35/lib/python3.5/s

In [ ]:
import tensorflow as tf

num_epochs = 10
batch_size = 30
movie_pix, movie_chunk_len = movie.shape[1], movie.shape[2]
n_layer = 3
n_neuron = frhist.shape[2]


def RNN(x):
    rnn_layers = [tf.nn.rnn_cell.LSTMCell(size) for size in [100, 100, 100]]
    
    lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)
    outputs, states = tf.nn.dynamic_rnn(lstm_cell, x, dtype=tf.float32)

input_frame = tf.placeholder(tf.float32, shape=(movie_chunk_len, movie_pix))
true_hist = tf.placeholder(tf.float32, shape=(movie_chunk_len, n_neuron))
pred_hist = RNN(input_frame)
cost = tf.keras.losses.poisson(true_hist, pred_hist)

In [5]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14975165826730809756
]
